In [30]:
from dask.distributed import Client

ip = "3.86.56.34"
address = f"tcp://{ip}:8786"
dashboard = f"http://{ip}:8787/status"

print(f"Use the link below to connect to the cluster dashboard:\n{dashboard}")

print(address)
client = Client(address=address)

client

Use the link below to connect to the cluster dashboard:
http://3.86.56.34:8787/status
tcp://3.86.56.34:8786


C:\Users\sacha\anaconda3\envs\dask-cluster\lib\site-packages\distributed\client.py:1606: VersionMismatchWarning: Mismatched versions found

+-------------+--------+-----------+---------+
| Package     | Client | Scheduler | Workers |
+-------------+--------+-----------+---------+
| cloudpickle | 3.0.0  | 3.1.0     | 3.1.0   |
+-------------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://172.31.33.167:8786' processes=3 threads=3, memory=2.80 GiB>

In [31]:
import dask.array as da

a_da = da.ones(10, chunks=5)
a_da

dask.array<ones_like, shape=(10,), dtype=float64, chunksize=(5,), chunktype=numpy.ndarray>

In [32]:
a_da_sum = a_da.sum()
a_da_sum

dask.array<sum-aggregate, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>

In [33]:
a_da_sum.compute()

np.float64(10.0)

In [36]:
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
xd

dask.array<normal, shape=(30000, 30000), dtype=float64, chunksize=(3000, 3000), chunktype=numpy.ndarray>

In [37]:
%%time
xd = da.random.normal(10, 0.1, size=(30_000, 30_000), chunks=(3000, 3000))
yd = xd.mean(axis=0)
yd.compute()

CPU times: total: 15.6 ms
Wall time: 9.07 s


array([ 9.99983712,  9.99983432,  9.99965233, ..., 10.00064962,
        9.99959245, 10.00039006])

In [38]:
!aws s3 ls

2024-10-11 18:26:55 cf-templates-r78qtn9pvstf-us-east-1
2024-10-11 19:24:19 dask-input-data


In [25]:
import dask.dataframe as dd

filenames = ["s3://dask-input-data/1990.csv", "s3://dask-input-data/1991.csv"]

# Example: Reading a CSV file from S3
ddf = dd.read_csv(filenames, 
                  parse_dates={"Date": [0, 1, 2]},
                  dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
                  blocksize="10MB" )

ddf

,Date,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=4,,,,,,,,,,,,,,,,,,,,,
,datetime64[ns],int64,float64,int64,float64,int64,string,int64,string,float64,float64,float64,float64,float64,string,string,float64,float64,float64,bool,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [44]:
import dask.dataframe as dd

# Read all CSV files from the root of the bucket
ddf = dd.read_csv("s3://dask-input-data/*.csv", 
                  dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
                  blocksize="25MB" )


ddf

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
npartitions=9,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int64,int64,float64,int64,float64,int64,string,int64,string,float64,float64,float64,float64,float64,string,string,float64,float64,float64,bool,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [40]:
%%time
len(ddf)

CPU times: total: 0 ns
Wall time: 5.55 s


2342711

In [41]:
ddf.head(2)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,Diverted
0,1990,1,1,1,1621.0,1540,1747.0,1701,US,33,...,NaN,46.0,41.0,EWR,PIT,319.0,NaN,NaN,False,0
1,1990,1,2,2,1547.0,1540,1700.0,1701,US,33,...,NaN,-1.0,7.0,EWR,PIT,319.0,NaN,NaN,False,0


In [42]:
%%time
result = ddf.DepDelay.max()
result.compute()

CPU times: total: 0 ns
Wall time: 3.89 s


np.float64(1435.0)

In [43]:
len(ddf[~ddf.Cancelled])

2284123

In [46]:
%%time
ddf[~ddf.Cancelled].groupby("Origin")["Origin"].count().compute()

CPU times: total: 15.6 ms
Wall time: 3.26 s


Origin
EWR    1019405
JFK     383399
LGA     881319
Name: Origin, dtype: int64

In [49]:
%%time
ddf.groupby("Origin").DepDelay.mean().compute()

CPU times: total: 15.6 ms
Wall time: 3.32 s


Origin
EWR    10.103120
JFK    10.511871
LGA     7.112117
Name: DepDelay, dtype: float64

In [50]:
%%time
ddf.groupby("DayOfWeek").DepDelay.mean().idxmax().compute()

CPU times: total: 31.2 ms
Wall time: 3 s


np.int64(5)

In [29]:
client.close()